# 8장, 데이터 wrangling: join, combine, reshape

https://wesmckinney.com/book/data-wrangling

In [1]:
!pip install -Uqq numpy pandas

In [2]:
import numpy as np
import pandas as pd

## 8.1 계층적 인덱싱

### 재정렬 및 ​​정렬 수준


### 레벨별 요약 통계


### DataFrame의 열을 사용한 인덱싱


## 8.2 데이터 세트 결합 및 병합

https://wesmckinney.com/book/data-wrangling#prep_merge_join

- pandas.merge
  - 하나 이상의 키를 기준으로 DataFrame의 행을 연결합니다. 이는 데이터베이스 `join` 연산을 구현하므로 SQL이나 다른 관계형 데이터베이스 사용자에게 익숙할 것입니다.
- pandas.concat
  - 축을 따라 객체를 연결하거나 "쌓습니다".
- combine_first
  - 겹치는 데이터를 이어 붙여 한 객체의 누락된 값을 다른 객체의 값으로 채웁니다.


### 데이터베이스 스타일 DataFrame 조인

병합 또는 조인 연산은 하나 이상의 키를 사용하여 행을 연결하여 데이터 세트를 결합한다.

```python
def merge(
    left: DataFrame | Series,
    right: DataFrame | Series,
    how: MergeHow = ...,
    on: Label | AnyArrayLike | list[HashableT@merge] = ...,
    left_on: Label | AnyArrayLike | list[HashableT@merge] = ...,
    right_on: Label | AnyArrayLike | list[HashableT@merge] = ...,
    left_index: bool = ...,
)
```

In [4]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")
                   })

df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")
                   })
df1, df2

(  key  data1
 0   b      0
 1   b      1
 2   a      2
 3   c      3
 4   a      4
 5   a      5
 6   b      6,
   key  data2
 0   a      0
 1   b      1
 2   d      2)

In [ ]:
pd.merge(df1, df2)
# 다대일 조인의 예.  어떤 컬럼으로 join 할 것인지 지정하지 않았음. 그럴 경우, 중복되는 컬럼 이름을 자동으로 선택함.

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [6]:
pd.merge(df1, df2, on='key') # 'key' 컬럼으로 서로 join.

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


### 인덱스 병합


In [8]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                     "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

left1, right1

(  key  value
 0   a      0
 1   b      1
 2   a      2
 3   a      3
 4   b      4
 5   c      5,
    group_val
 a        3.5
 b        7.0)

In [9]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


pandas.merge 함수의 인자

- ...
- how
  - 적용 할 조인 유형
  - "inner", "outer", "left", "right" 중 하나. 기본값은 "inner".
- left_on / right_on
  - join 키로 사용할 left/right DataFrame 의 열. 단일 열 이름, 또는 열 이름 리스트.
- left_index / right_index
  - left/right 의 행 인덱스를 조인 키로 사용. (만약 MultiIndex 라면 여러 키)
- ...

In [4]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index=["a", "c", "e"],
                    columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                    index=["b", "c", "d", "e"],
                    columns=["Missouri", "Alabama"]).astype("Int64")

left2, right2

(   Ohio  Nevada
 a     1       2
 c     3       4
 e     5       6,
    Missouri  Alabama
 b         7        8
 c         9       10
 d        11       12
 e        13       14)

In [5]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


...

- DataFrame 에는 인덱스별 병합을 간소화하는 `join` 인스턴스 메서드가 있습니다. 
- 또한, 동일하거나 유사한 인덱스를 갖지만 열이 겹치지 않는 여러 DataFrame 객체를 결합하는 데에도 사용할 수 있습니다. 
- 이전 예제에서는 다음과 같이 작성할 수 있었습니다.


In [6]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [ ]:
left1.join(right1, on="key")

### 축을 따라 연결

### 오버랩을 이용한 데이터 결합

## 8.3 재구성 및 피벗

### 계층적 인덱싱을 사용한 재구성

### "긴" 포맷에서 "넓은" 포맷으로 전환

### "와이드" 포맷에서 "롱" 포맷으로 전환